# Projecte Clustering FEB  
## Part 1 – ETL, EDA i model de dades

Objectiu: transformar les dades crues de MongoDB en un dataset net, coherent i preparat per aplicar models de clustering no supervisat.

In [34]:
# Imports bàsics
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

## Connexió a MongoDB

In [35]:
# Connexió MongoDB (configuració)
from pymongo import MongoClient
from config import MONGODB_URI, DATABASE_NAME

## Càrrega de dades (filtrat selectiu)

**Consulta selectiva per temporada i competició:**

In [36]:
# Carregar dades de MongoDB amb filtrat selectiu
print("🔄 Carregant dades de MongoDB...")

client = MongoClient(MONGODB_URI, serverSelectionTimeoutMS=30000)
db = client[DATABASE_NAME]
collection = db["FEB3_players_statistics"]

# Primer veiem quines temporades i competicions existeixen
print("📋 Explorant dades disponibles...")
sample = list(collection.find().limit(5))
if sample:
    print("Columnes disponibles:", list(sample[0].keys()))
    
    # Veure temporades úniques (usant season_id)
    seasons = collection.distinct("season_id")
    print(f"Temporades disponibles: {seasons[:5]}...")  # Primeres 5
    
    # Veure competicions úniques
    competitions = collection.distinct("competition_feb_id")
    print(f"Competicions disponibles: {competitions[:10]}...")  # Primeres 10

# Carregar totes les dades (no hi ha temporada 2024-2025)
df = pd.DataFrame(list(collection.find().limit(10000)))

print(f"✅ {len(df):,} registres carregats")
if len(df) > 0:
    print(f"🏀 {df['player_feb_id'].nunique():,} jugadors únics")
    print(f"📊 Temporades (season_id): {df['season_id'].unique()}")
    print(f"🏆 Competicions: {df['competition_feb_id'].unique()[:5]}...")

🔄 Carregant dades de MongoDB...
📋 Explorant dades disponibles...
Columnes disponibles: ['competition_feb_id', 'competition_name', 'season_id', 'phase_feb_id', 'phase_feb_name', 'subphase_feb_id', 'subphase_feb_name', 'round_feb_id', 'round_feb_name1', 'round_feb_name2', 'match_feb_id', 'data', 'hora', 'local', '2pa', 'pps', 'pfd', 'rc_pl_m', 'rc_ce3r_m', 'pts', 'fg', 'rc_pr_a', 'rc_c3l_m', 'ts', 'ast2p', 'rc_mel_a', 'team_feb_code', 'tsa', 'astfd', 'orb', 'ftr', 'starter', '2pm', 'rc_mbl_m', 'fgm', 'fga', 'ast', 'pf', 'eff_spanish', 'balance', 'rc_mer_m', 'rc_ce3l_m', 'team_feb_id', 'ftm', 'ppt2', 'blka', 'rc_e3l_a', 'team_club_feb_code', 'rc_mbl_a', 'rc_e3r_m', 'ast3p', 'drb', 'rc_mel_m', '3pa', 'ppt3', 'tov', 'rc_ce3l_a', 'rc_c3r_m', 'rc_c3r_a', 'rc_ce3r_a', 'rc_pc_m', 'rc_mbr_m', 'minutes', 'rc_pc_a', 'rc_pr_m', 'rc_mer_a', 'rc_e3l_m', 'rc_pl_a', 'trb', 'rc_e3r_a', 'player_number', 'efg', 'rc_c3l_a', 'fta', 'stl', 'blk', '3pm', 'dunk', 'rc_mbr_a', 'team_name', 'player_feb_id', 'play

## Disseny del model de dades analític

**Nivell d'anàlisi:** Mitjanes per partit per jugador

**Variables rellevants per clustering:**
- **Ofensives:** pts, ast, tov, usage_2p, usage_3p
- **Eficiència:** pct_2p, pct_3p, pct_inside
- **Joc complet:** trb (rebots)

In [37]:
# Neteja de dades
print("🧹 Netejant dades...")

# 1. Tractament de valors nuls (només en columnes clau)
key_columns = ["player_feb_id", "pts", "ast", "trb", "tov", "minutes", "fga", "2pa", "3pa", "2pm", "3pm"]
print(f"Registres abans de neteja: {len(df):,}")

# Comprovar valors nuls en columnes clau
null_counts = df[key_columns].isnull().sum()
print(f"Valors nuls per columna: {dict(null_counts)}")

# Eliminar només files amb nuls en columnes CRÍTIQUES
critical_cols = ["player_feb_id", "pts", "minutes"]  # Mínim indispensable
df_clean = df.dropna(subset=critical_cols)
print(f"✅ Valors nuls crítics eliminats: {len(df) - len(df_clean)} registres")

# 2. Filtrar minuts = 0 (però ser menys restrictiu)
df_clean = df_clean[df_clean["minutes"] > 0]
print(f"✅ Minuts = 0 eliminats: {len(df_clean)} registres")

# 3. Filtrar jugadors amb poca mostra (REDUÏT a 2 partits per tenir més dades)
player_counts = df_clean["player_feb_id"].value_counts()
valid_players = player_counts[player_counts >= 2].index  # Canviat de 5 a 2
df_clean = df_clean[df_clean["player_feb_id"].isin(valid_players)]
print(f"✅ Jugadors amb <2 partits eliminats: {len(df_clean)} registres")

# 4. Convertir minuts a reals
df_clean["minutes"] = df_clean["minutes"] / 60

# 5. Eliminar columnes no rellevants
drop_cols = ["_id", "hora", "round_feb_name2", "subphase_feb_id", "subphase_feb_name"]
df_clean = df_clean.drop(columns=[c for c in drop_cols if c in df_clean.columns])

print(f"✅ Neteja completada: {len(df_clean):,} registres vàlids")
print(f"🏀 Jugadors únics: {df_clean['player_feb_id'].nunique():,}")

df = df_clean  # Actualitzar df per a les següents cel·les

🧹 Netejant dades...
Registres abans de neteja: 10,000
Valors nuls per columna: {'player_feb_id': np.int64(0), 'pts': np.int64(0), 'ast': np.int64(0), 'trb': np.int64(0), 'tov': np.int64(0), 'minutes': np.int64(0), 'fga': np.int64(0), '2pa': np.int64(0), '3pa': np.int64(0), '2pm': np.int64(0), '3pm': np.int64(0)}
✅ Valors nuls crítics eliminats: 0 registres
✅ Minuts = 0 eliminats: 9124 registres
✅ Jugadors amb <2 partits eliminats: 8978 registres
✅ Neteja completada: 8,978 registres vàlids
🏀 Jugadors únics: 1,044


## Creació de noves estadístiques avançades

In [38]:
# Crear noves variables estadístiques
print("⚙️ Creant features avançades...")

# 1. Ús de tirs de 2 i 3
df["usage_2p"] = df["2pa"] / (df["fga"] + 1e-6)
df["usage_3p"] = df["3pa"] / (df["fga"] + 1e-6)

# 2. Percentatges d'eficàcia
df["pct_2p"] = df["2pm"] / (df["2pa"] + 1e-6)
df["pct_3p"] = df["3pm"] / (df["3pa"] + 1e-6)

# 3. Eficàcia per zona de tir (interior vs exterior)
df["shots_inside"] = df["rc_pc_a"] + df["rc_pr_a"] + df["rc_pl_a"]
df["shots_outside"] = df["3pa"]
df["pct_inside"] = df["shots_inside"] / (df["shots_inside"] + df["shots_outside"] + 1e-6)

# 4. OER (Offensive Efficiency Rating) - Punts per 100 possessions
# Estimació de possessions: FGA - OR + TOV + 0.44*FTA
df["possessions"] = df["fga"] - df.get("or", 0) + df["tov"] + 0.44 * df.get("fta", 0)
df["oer"] = (df["pts"] / (df["possessions"] + 1e-6)) * 100

# 5. DER (Defensive Efficiency Rating) - Simplificat
# Com que no tenim opp_pts, fem una estimació basada en l'eficiència defensiva
df["der"] = 100 / (df["oer"] / 100 + 1e-6)  # Estimació inversa simple

print("✅ Features avançades creats:")
print("   - Ús de tirs (2p/3p)")
print("   - Eficàcia per zona (%)")
print("   - OER (Offensive Efficiency Rating)")
print("   - DER (Defensive Efficiency Rating - estimat)")

⚙️ Creant features avançades...
✅ Features avançades creats:
   - Ús de tirs (2p/3p)
   - Eficàcia per zona (%)
   - OER (Offensive Efficiency Rating)
   - DER (Defensive Efficiency Rating - estimat)


## Dataset final (agregació per jugador)

In [39]:
# Dataset final: mitjanes per partit per jugador
print("📊 Creant dataset final...")

# Variables rellevants per clustering
features_clustering = [
    "pts", "ast", "trb", "tov",           # Estadístiques bàsiques
    "usage_2p", "usage_3p",               # Ús de tirs
    "pct_2p", "pct_3p", "pct_inside",     # Eficàcia
    "oer", "der"                          # Efficiency ratings
]

# Agregar per jugador (mitjanes per partit)
df_player = df.groupby(["player_feb_id", "player_name"])[features_clustering].mean().reset_index()

print(f"✅ {len(df_player)} jugadors processats")
print(f"🎯 {len(features_clustering)} features per jugador")

# Justificació de variables escollides
print("\n📋 JUSTIFICACIÓ DE VARIABLES:")
print("• pts, ast, trb, tov: Producció ofensiva bàsica")
print("• usage_2p, usage_3p: Estil de joc i preferències de tir")
print("• pct_2p, pct_3p, pct_inside: Eficàcia en diferents zones")
print("• oer, der: Eficiència ofensiva i defensiva global")

# Guardar dataset final
df_player.to_csv('jugadors_processats.csv', index=False, encoding='utf-8-sig')
print("💾 Dataset final guardat com 'jugadors_processats.csv'")

df_player.head()

📊 Creant dataset final...
✅ 1050 jugadors processats
🎯 11 features per jugador

📋 JUSTIFICACIÓ DE VARIABLES:
• pts, ast, trb, tov: Producció ofensiva bàsica
• usage_2p, usage_3p: Estil de joc i preferències de tir
• pct_2p, pct_3p, pct_inside: Eficàcia en diferents zones
• oer, der: Eficiència ofensiva i defensiva global
💾 Dataset final guardat com 'jugadors_processats.csv'


,player_feb_id,player_name,pts,ast,trb,tov,usage_2p,usage_3p,pct_2p,pct_3p,pct_inside,oer,der
0,1042219,R. GUARDIA MARTI,2.333333,0.666667,2.000000,1.000000,0.538889,0.461111,0.500000,0.083333,0.233333,40.213170,3.333344e+07
1,1068539,D. SANCHEZ MORDOS,7.916667,1.250000,4.500000,1.000000,0.350983,0.649017,0.402778,0.358532,0.283432,94.102408,8.333435e+06
2,1069568,M. BLANCH ESPINAL,10.285714,0.714286,1.000000,1.000000,0.464430,0.535570,0.642857,0.278571,0.389796,91.431046,1.241862e+02
3,1074113,J. GALVE NAVARRO,7.333333,2.166667,1.166667,0.833333,0.196429,0.803571,0.250000,0.388889,0.148809,104.261493,1.165255e+02
4,1074329,C. MARZO CHECA,7.333333,4.000000,3.666667,1.666667,0.571428,0.428571,0.633333,0.166667,0.555555,93.305715,1.415553e+02
